<a href="https://colab.research.google.com/github/samitha278/multilayer-perceptron-char-lm/blob/main/mlp_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# download the names.txt file from github
!wget https://github.com/samitha278/multilayer-perceptron-char-lm/blob/main/data/names.txt

--2025-08-23 06:35:35--  https://github.com/samitha278/multilayer-perceptron-char-lm/blob/main/data/names.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘names.txt’

names.txt               [ <=>                ] 560.03K  --.-KB/s    in 0.1s    

2025-08-23 06:35:36 (3.71 MB/s) - ‘names.txt’ saved [573473]



In [3]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

2090
392863
['', '', '', '', '', '', '<!DOCTYPE html>', '<html']


In [4]:
#character mapping
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: ' ', 2: '!', 3: '"', 4: '#', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 0: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '<', 29: '=', 30: '>', 31: '?', 32: '@', 33: 'A', 34: 'B', 35: 'C', 36: 'D', 37: 'E', 38: 'F', 39: 'G', 40: 'H', 41: 'I', 42: 'J', 43: 'K', 44: 'L', 45: 'M', 46: 'N', 47: 'O', 48: 'P', 49: 'Q', 50: 'R', 51: 'S', 52: 'T', 53: 'U', 54: 'V', 55: 'W', 56: 'X', 57: 'Y', 58: 'Z', 59: '[', 60: ']', 61: '_', 62: '`', 63: 'a', 64: 'b', 65: 'c', 66: 'd', 67: 'e', 68: 'f', 69: 'g', 70: 'h', 71: 'i', 72: 'j', 73: 'k', 74: 'l', 75: 'm', 76: 'n', 77: 'o', 78: 'p', 79: 'q', 80: 'r', 81: 's', 82: 't', 83: 'u', 84: 'v', 85: 'w', 86: 'x', 87: 'y', 88: 'z', 89: '{', 90: '}', 91: '·', 92: '’', 93: '…'}
93
